In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, SubsetRandomSampler, DataLoader
from torchvision.datasets import MNIST
from torchvision.models import resnet18
from tqdm import tqdm

# Define transforms for dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

# Define initial labeled dataset
num_initial_samples = 1000
labeled_indices = torch.randperm(len(train_dataset))[:num_initial_samples]
labeled_dataset = torch.utils.data.Subset(train_dataset, labeled_indices)

# Define unlabeled dataset
# unlabeled_indices = torch.arange(len(train_dataset))[~torch.isin(torch.arange(len(train_dataset)), labeled_indices)]
# unlabeled_dataset = torch.utils.data.Subset(train_dataset, unlabeled_indices)
unlabeled_indices = torch.arange(len(train_dataset))[~torch.eq(torch.arange(len(train_dataset)).unsqueeze(1), labeled_indices).any(1)]
unlabeled_dataset = torch.utils.data.Subset(train_dataset, unlabeled_indices)


# Define data loaders
labeled_loader = DataLoader(labeled_dataset, batch_size=32, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

import torch.nn as nn
from torchvision.models import resnet18

# Define ResNet model
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=True)
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.resnet18.conv1(x)
        x = self.resnet18.bn1(x)
        x = self.resnet18.relu(x)
        x = self.resnet18.maxpool(x)

        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)

        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = ResNet18(num_classes=10)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train function
def train(model, optimizer, labeled_loader):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    total = 0
    for images, labels in labeled_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_acc += (predicted == labels).sum().item()
    train_loss /= len(labeled_loader)
    train_acc /= total
    return train_loss, train_acc

In [33]:
# Active learning loop
num_queries = 10
batch_size = 32

for query in range(num_queries):
    print("Query number:", query+1)
    
    # Train model on labeled dataset
    train_loss, train_acc = train(model, optimizer, labeled_loader)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

    # Evaluate model on test dataset
    with torch.no_grad():
        test_loss = 0.0
        test_acc = 0.0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            test_acc += (predicted == labels).sum().item()
        test_loss /= len(test_loader)
        test_acc /= total
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    
    # Make model predictions on unlabeled dataset
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in unlabeled_loader:
            output = model(images)
            predictions.append(output)
    predictions = torch.cat(predictions, dim=0)

    # Select samples to label using entropy-based method
    entropy = -torch.sum(torch.softmax(predictions, dim=1) * torch.log(torch.softmax(predictions, dim=1)), dim=1)
    _, idx = torch.topk(entropy, k=batch_size)
    labeled_indices = torch.cat([labeled_indices, unlabeled_indices[idx]])
    unlabeled_indices = unlabeled_indices[~torch.eq(unlabeled_indices.unsqueeze(1), unlabeled_indices[idx]).any(1)]
    
    # Update labeled and unlabeled datasets
    labeled_dataset = torch.utils.data.Subset(train_dataset, labeled_indices)
    unlabeled_dataset = torch.utils.data.Subset(train_dataset, unlabeled_indices)

    # Update labeled and unlabeled data loaders
    labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)
    unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)

Query number: 1
Train Loss: 0.3774, Train Accuracy: 0.9090
Test Loss: 0.2743, Test Accuracy: 0.9296
Query number: 2
Train Loss: 0.2224, Train Accuracy: 0.9424
Test Loss: 0.2172, Test Accuracy: 0.9492
Query number: 3
Train Loss: 0.3087, Train Accuracy: 0.9353
Test Loss: 0.2801, Test Accuracy: 0.9289
Query number: 4
Train Loss: 0.2488, Train Accuracy: 0.9413
Test Loss: 0.2396, Test Accuracy: 0.9452
Query number: 5
Train Loss: 0.2781, Train Accuracy: 0.9371
Test Loss: 0.2326, Test Accuracy: 0.9402
Query number: 6
Train Loss: 0.2036, Train Accuracy: 0.9475
Test Loss: 0.2262, Test Accuracy: 0.9492
Query number: 7
Train Loss: 0.3066, Train Accuracy: 0.9356
Test Loss: 0.2141, Test Accuracy: 0.9498
Query number: 8
Train Loss: 0.1741, Train Accuracy: 0.9538
Test Loss: 0.1824, Test Accuracy: 0.9566
Query number: 9
Train Loss: 0.1664, Train Accuracy: 0.9601
Test Loss: 0.1720, Test Accuracy: 0.9601
Query number: 10
Train Loss: 0.1217, Train Accuracy: 0.9711
Test Loss: 0.1705, Test Accuracy: 0.9573